<a href="https://colab.research.google.com/github/Soob00/20251R0136COSE47101/blob/dataset2_pattern-mining/patternming/notebooks/dataset2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
import pandas as pd

file_path = '/content/drive/MyDrive/2025 Data Science Project/원핫인코딩_결측치가 포함된 모든 행 제거.csv'

data = pd.read_csv(file_path, encoding="utf-8")

# 불러온 csv 파일 읽기
print(data.head())

   YEAR       AGE  EDUC  GENDER  SPHSERVICE  CMPSERVICE  OPISERVICE  \
0  2022  0.461538  0.50       1           0           1           0   
1  2022  1.000000  0.75       0           1           1           0   
2  2022  0.615385  1.00       0           0           1           1   
3  2022  0.615385  1.00       0           0           1           1   
4  2022  0.846154  0.50       0           0           1           0   

   RTCSERVICE  IJSSERVICE  MH1  ...  LIVARAG_3  DIVISION_1  DIVISION_2  \
0           0           0    7  ...      False       False       False   
1           0           0   11  ...       True       False       False   
2           0           0    7  ...      False       False       False   
3           0           0    7  ...      False       False       False   
4           0           0    6  ...      False       False       False   

   DIVISION_3  DIVISION_4  DIVISION_5  DIVISION_6  DIVISION_7  DIVISION_8  \
0       False       False       False        True  

# **STEP1: 질환 간의 동시 발생 패턴 탐색**
예: "우울증이 있는 사람은 불안장애도 함께 겪는 경우가 많다"

방법: FP-Growth로 질환 간 조합 찾기**

- **사용 칼럼**: TRAUSTRE, ANXIETY, ADHD, CONDUCT, DELIRDEM, BIPOLAR, DEPRESS, ODD, PDD, PERSON, SCHIZO, ALCSUB, OTHERDIS, SUB.
- **이유**: 이 변수들은 질병의 존재 여부를 나타내며, 질환 간 동시 발생 패턴(예: DEPRESS와 ANXIETY의 조합)을 찾는 데 적합.





---



In [8]:
from mlxtend.frequent_patterns import fpgrowth, association_rules

# 1. 결측값이 있는 행 제거
data_cleaned = data.dropna()

# 2. 질병 변수만 추출
disease_columns = ['TRAUSTREFLG', 'ANXIETYFLG', 'ADHDFLG', 'CONDUCTFLG', 'DELIRDEMFLG', 'BIPOLARFLG', 'DEPRESSFLG', 'ODDFLG', 'PDDFLG', 'PERSONFLG', 'SCHIZOFLG', 'ALCSUBFLG', 'OTHERDISFLG', 'SUB_1', 'SUB_2', 'SUB_3', 'SUB_4', 'SUB_5', 'SUB_6', 'SUB_7', 'SUB_8', 'SUB_9', 'SUB_10', 'SUB_11', 'SUB_12', 'SUB_13']
disease_data = data_cleaned[disease_columns]

# FP-Growth 알고리즘 적용하여 자주 등장하는 항목 집합 찾기
# 3. minimum support: 0.01
frequent_itemsets = fpgrowth(disease_data, min_support=0.01, use_colnames=True)

# 4. minimum lift(향상도) 1.2 이상인 연관 규칙 생성
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.2)

# 5. 결과 출력
print("Frequent Itemsets:")
print(frequent_itemsets)
print("\nAssociation Rules:")
print(rules)

/usr/local/lib/python3.11/dist-packages/mlxtend/frequent_patterns/fpcommon.py:161: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


Frequent Itemsets:
     support                               itemsets
0   0.350759                           (DEPRESSFLG)
1   0.159832                            (SCHIZOFLG)
2   0.240187                           (ANXIETYFLG)
3   0.190046                          (TRAUSTREFLG)
4   0.015345                               (SUB_11)
5   0.159783                           (BIPOLARFLG)
6   0.025109                                (SUB_8)
7   0.113109                          (OTHERDISFLG)
8   0.042193                                (SUB_4)
9   0.036234                            (PERSONFLG)
10  0.050475                              (ADHDFLG)
11  0.018011                                (SUB_6)
12  0.011454                                (SUB_9)
13  0.027110                                (SUB_7)
14  0.016260                               (PDDFLG)
15  0.064401                            (ALCSUBFLG)
16  0.010344                (SCHIZOFLG, DEPRESSFLG)
17  0.016268                (SCHIZOFLG, ANXIE

# FP-Growth 분석 결과 확인

**빈번한 질환 조합 (Frequent Itemsets)**
  (최소 지지도(1%) 이상인 조합들만 나열함)

- 가장 빈번한 단일 질환: DEPRESSFLG (우울증 관련 질환)이 35.08%의 지지도(support)로 가장 자주 나타남
- 주요 2개 조합: DEPRESSFLG와 ANXIETYFLG (불안 관련 질환)의 조합이 9.84%로 가장 흔한 쌍임
- 3개 조합: DEPRESSFLG, TRAUSTREFLG (외상 관련 질환), ANXIETYFLG가 1.61%의 지지도를 보임

**연관 규칙 (Association Rules)**
  (모든 규칙은 리프트 1.2 이상으로 필터링되어, 의미 있는 연관성만 포함함)
- 규칙 1: DEPRESSFLG와 TRAUSTREFLG가 있을 때 ANXIETYFLG가 나타날 확률(confidence)은 30.39%, 리프트(lift)는 1.27로 양의 상관관계를 보임
- 규칙 2: TRAUSTREFLG와 ANXIETYFLG가 있을 때 DEPRESSFLG가 나타날 확률은 45.58%, 리프트는 1.30으로 강한 연관성을 시사함

**CONCLUSION: DEPRESSFLG, ANXIETYFLG, TRAUSTREFLG 같은 질환들이 자주 함께 나타남**



---
# **STEP 2: 질환+배경 변수의 연관 규칙**
 **사용 칼럼**:
 - 질환: 위와 동일
 - 배경: AGE, EDUC, ETHNIC, RACE, GENDER, MARSTAT, SMISED, SAP, EMPLOY, DETNLF, VETERAN, LIVARAG.

- **이유**: 질환과 배경 변수 간 연관성을 찾기 위해 모두 포함(예: "DEPRESS + EMPLOY=실업 → PTSD").
- **방법**: FP-Growth

In [11]:
# 1. 결측값이 있는 행 제거
data_cleaned = data.dropna()

# 2. 배경 변수 추출
background_columns = [
    "AGE", "EDUC", "GENDER", "SPHSERVICE", "CMPSERVICE", "OPISERVICE", "RTCSERVICE", "IJSSERVICE",
    "RACE_2", "RACE_3", "RACE_4", "RACE_5", "RACE_6",
    "ETHNIC_2", "ETHNIC_3", "ETHNIC_4",
    "MARSTAT_2", "MARSTAT_3", "MARSTAT_4",
    "SMISED_2", "SMISED_3",
    "DETNLF_2", "DETNLF_3", "DETNLF_4", "DETNLF_5", "DETNLF_NotApplicable",
    "LIVARAG_2", "LIVARAG_3"
]

background_data = data_cleaned[background_columns]

# Combine all columns
all_columns = disease_data + background_data

# Extract relevant data
data_all = df[all_columns]

# Check for NaN values
print("NaN counts before processing:")
print(data_all.isna().sum())

# Handle AGE (binning)
data_all['AGE_bin_0_18'] = (data_all['AGE'] <= 18).astype(int)
data_all['AGE_bin_19_30'] = ((data_all['AGE'] > 18) & (data_all['AGE'] <= 30)).astype(int)
data_all['AGE_bin_31_50'] = ((data_all['AGE'] > 30) & (data_all['AGE'] <= 50)).astype(int)
data_all['AGE_bin_51_plus'] = (data_all['AGE'] > 50).astype(int)

# Update columns to exclude AGE and include binned AGE
all_columns = [col for col in all_columns if col != 'AGE'] + ['AGE_bin_0_18', 'AGE_bin_19_30', 'AGE_bin_31_50', 'AGE_bin_51_plus']
data_all = data_all[all_columns]

# NaN 은 0 대체
data_all = data_all.fillna(0)

# Verify no NaN values remain
print("\nNaN counts after processing:")
print(data_all.isna().sum())

# Check number of rows
print("\nNumber of rows in data_all:", len(data_all))

# Ensure data is binary
print("\nUnique values in each column:")
for col in data_all.columns:
    print(f"{col}: {data_all[col].unique()}")

# Apply FP-Growth
if len(data_all) > 0:
    frequent_itemsets = fpgrowth(data_all, min_support=0.01, use_colnames=True)

    # Generate association rules
    rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.2)

    # Filter rules
    disease_to_background_rules = rules[rules['antecedents'].apply(lambda x: any(item in disease_columns for item in x)) &
                                        rules['consequents'].apply(lambda x: any(item in background_columns for item in x))]
    background_to_disease_rules = rules[rules['antecedents'].apply(lambda x: any(item in background_columns for item in x)) &
                                        rules['consequents'].apply(lambda x: any(item in disease_columns for item in x))]

    # Output results
    print("\nDisease to Background Rules:")
    print(disease_to_background_rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']])
    print("\nBackground to Disease Rules:")
    print(background_to_disease_rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']])

    # Save rules
    disease_to_background_rules.to_csv('disease_to_background_rules.csv', index=False)
    background_to_disease_rules.to_csv('background_to_disease_rules.csv', index=False)
else:
    print("Error: DataFrame is empty after processing.")


NaN counts before processing:
YEAR          1147781
AGE           1147781
EDUC          1147781
GENDER        1147781
SPHSERVICE    1147781
               ...   
DIVISION_5    1147781
DIVISION_6    1147781
DIVISION_7    1147781
DIVISION_8    1147781
DIVISION_9    1147781
Length: 77, dtype: int64


<ipython-input-11-08354c3afa09>:38: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data_all = data_all.fillna(0)



NaN counts after processing:
ADHDFLG                 0
ALCSUBFLG               0
ANXIETYFLG              0
BIPOLARFLG              0
CMPSERVICE              0
CONDUCTFLG              0
DELIRDEMFLG             0
DEPRESSFLG              0
DETNLF_2                0
DETNLF_3                0
DETNLF_4                0
DETNLF_5                0
DETNLF_NotApplicable    0
EDUC                    0
ETHNIC_2                0
ETHNIC_3                0
ETHNIC_4                0
GENDER                  0
IJSSERVICE              0
LIVARAG_2               0
LIVARAG_3               0
MARSTAT_2               0
MARSTAT_3               0
MARSTAT_4               0
ODDFLG                  0
OPISERVICE              0
OTHERDISFLG             0
PDDFLG                  0
PERSONFLG               0
RACE_2                  0
RACE_3                  0
RACE_4                  0
RACE_5                  0
RACE_6                  0
RTCSERVICE              0
SCHIZOFLG               0
SMISED_2                0
SMISED_3

/usr/local/lib/python3.11/dist-packages/mlxtend/frequent_patterns/fpcommon.py:161: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


ValueError: The input DataFrame `df` containing the frequent itemsets is empty.

# step 2에서 결측치 때문에 계속 오류가 나는데 아직 왜 그런지 정확하게 파악을 못했어요 그래서 여기서 부터 밑에 까지는 아직 결과 분석을 step 1 처럼 정리 못했음

# **Step 3: 전략 B - 질환 행렬로 K-Means 클러스터링**

- **사용 데이터**: disease data
- **이유**: 질환 행렬(바이너리 데이터)을 사용해 환자를 질병 프로필로 군집화.


In [12]:
import pandas as pd
from sklearn.cluster import KMeans

# K-평균 클러스터링 실행 (초기 클러스터 개수는 5개, 데이터에 따라 조정 가능)
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
clusters = kmeans.fit_predict(disease_data)

# 클러스터 레이블을 데이터프레임에 추가
df['cluster'] = clusters

# 각 클러스터별 질병 변수의 평균으로 요약
cluster_summary = df.groupby('cluster')[disease_columns].mean()
print("Cluster Summary (Mean of Disease Variables):")
print(cluster_summary)

# 클러스터 레이블이 포함된 데이터프레임 저장n_clusters = 5
df.to_csv('clustered_data.csv', index=False)

Cluster Summary (Mean of Disease Variables):
         TRAUSTREFLG  ANXIETYFLG   ADHDFLG  CONDUCTFLG  DELIRDEMFLG  \
cluster                                                               
0           0.358203    0.000000  0.087998    0.016742     0.005432   
1           0.150272    0.293254  0.033993    0.001979     0.002143   
2           0.067806    0.085581  0.014430    0.001988     0.005166   
3           0.158265    0.000000  0.048286    0.002980     0.002242   
4           0.137748    1.000000  0.064578    0.003425     0.001682   

         BIPOLARFLG  DEPRESSFLG    ODDFLG    PDDFLG  PERSONFLG  ...     SUB_4  \
cluster                                                         ...             
0          0.000000    0.000000  0.019130  0.035716   0.027879  ...  0.029243   
1          0.009419    1.000000  0.002974  0.005925   0.032951  ...  0.050392   
2          0.028642    0.051650  0.001178  0.009175   0.037019  ...  0.040548   
3          1.000000    0.058911  0.003303  0.008384 

# **Step 4: 전략 B - 클러스터별 배경 조건 확인**

- **사용 칼럼**: AGE, EDUC, ETHNIC, RACE, GENDER, MARSTAT, SMISED, SAP, EMPLOY, DETNLF, VETERAN, LIVARAG.
- **방법 제안**:
    - 클러스터별로 배경 변수의 분포를 계산(예: 평균 AGE, EMPLOY 상태 비율).
    - 시각화(막대그래프, 히스토그램)로 각 클러스터의 배경 특성 비교.
    - 통계 검정(카이제곱, t-검정)으로 클러스터 간 차이 확인 가능.
- **이유**: 질병 기반 클러스터에서 배경 변수의 패턴을 역으로 탐색.

In [13]:
# 클러스터별 배경 변수의 비율 분석
print("Background Variable Proportions by Cluster:")
for col in background_columns:
    cluster_col_summary = df.groupby('cluster')[col].mean()
    print(f"\nProportion of {col} in each cluster:")
    print(cluster_col_summary)


with open('cluster_background_summary.txt', 'w') as f:
    for col in background_columns:
        cluster_col_summary = df.groupby('cluster')[col].mean()
        f.write(f"\nProportion of {col} in each cluster:\n{cluster_col_summary}\n")

Background Variable Proportions by Cluster:

Proportion of AGE in each cluster:
cluster
0    0.511879
1    0.573901
2    0.646803
3    0.601051
4    0.541686
Name: AGE, dtype: float64

Proportion of EDUC in each cluster:
cluster
0    0.679055
1    0.736945
2    0.701788
3    0.750926
4    0.744719
Name: EDUC, dtype: float64

Proportion of GENDER in each cluster:
cluster
0    0.507266
1    0.384489
2    0.628486
3    0.407361
4    0.378856
Name: GENDER, dtype: float64

Proportion of SPHSERVICE in each cluster:
cluster
0    0.003674
1    0.005117
2    0.019713
3    0.008018
4    0.003879
Name: SPHSERVICE, dtype: float64

Proportion of CMPSERVICE in each cluster:
cluster
0    0.960403
1    0.951717
2    0.938577
3    0.949873
4    0.984044
Name: CMPSERVICE, dtype: float64

Proportion of OPISERVICE in each cluster:
cluster
0    0.065429
1    0.066721
2    0.101899
3    0.078855
4    0.021402
Name: OPISERVICE, dtype: float64

Proportion of RTCSERVICE in each cluster:
cluster
0    0.014804
1

# **Step 5: 전략 A와 B 비교**

- **사용 칼럼**: Step 2와 Step 3~4의 결과 활용.
- **방법**:
    - 전략 A(연관 규칙)에서 찾은 조합(예: DEPRESS+무직 → PTSD)과 전략 B(클러스터)의 질병+배경 패턴을 비교.
    - 일치 여부는 공통된 질병 조합과 배경 변수의 유사성으로 판단.

In [ ]:
# 클러스터 요약 정보 다시 계산
cluster_summary = df.groupby('cluster')[disease_columns].mean()

# 예시 비교: DEPRESSFLG와 ANXIETYFLG 값이 높은 클러스터가 SMISED_2와 관련된 연관 규칙과 일치하는지 확인
print("Comparing Strategy A (Rules) with Strategy B (Clusters):")
high_depress_anxiety_clusters = cluster_summary[(cluster_summary['DEPRESSFLG'] > 0.5) & (cluster_summary['ANXIETYFLG'] > 0.5)].index

if not high_depress_anxiety_clusters.empty:
    print("\nClusters with high DEPRESSFLG and ANXIETYFLG:")
    for cluster in high_depress_anxiety_clusters:
        smised_2_proportion = df[df['cluster'] == cluster]['SMISED_2'].mean()
        print(f"Cluster {cluster}: Proportion of SMISED_2 = {smised_2_proportion}")
else:
    print("\nNo clusters found with high DEPRESSFLG and ANXIETYFLG.")

# 전략 A에서 생성된 규칙 중 이 패턴과 일치하는 것이 있는지 확인
relevant_rules = background_to_disease_rules[
    background_to_disease_rules['antecedents'].str.contains('SMISED_2') &
    background_to_disease_rules['consequents'].str.contains('DEPRESSFLG|ANXIETYFLG')
]
print("\nRelevant Rules from Strategy A (SMISED_2 -> DEPRESSFLG or ANXIETYFLG):")
if not relevant_rules.empty:
    print(relevant_rules[['antecedents', 'consequents', 'lift']])
else:
    print("No matching rules found.")